# Vectorizing Raw Data: N-Grams

### N-Grams 

Creates a document-term matrix where counts still occupy the cell but instead of the columns representing single terms, they represent all combinations of adjacent words of length n in your text.

"NLP is an interesting topic"

| n | Name      | Tokens                                                         |
|---|-----------|----------------------------------------------------------------|
| 2 | bigram    | ["nlp is", "is an", "an interesting", "interesting topic"]      |
| 3 | trigram   | ["nlp is an", "is an interesting", "an interesting topic"] |
| 4 | four-gram | ["nlp is an interesting", "is an interesting topic"]    |

### Read in text

In [ ]:
import pandas as pd
import re
import string
import nltk
pd.set_option('display.max_colwidth', 100)

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

### Create function to remove punctuation, tokenize, remove stopwords, and stem

In [ ]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    # difference here: we need to join tokens together to reconstruct back to a string
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    return text

data['cleaned_text'] = data['body_text'].apply(lambda x: clean_text(x))
data.head()

### Apply CountVectorizer (w/ N-Grams)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# instantiate the count vectorizer like before, except  we don't need an analyzer here
#   instead we need ngram, and we want to only look at bigrams
ngram_vec = CountVectorizer(ngram_range=(2,2))
x_counts = ngram_vec.fit_transform(data['cleaned_text'])

# and just like before:
print(x_counts.shape)
print(ngram_vec.get_feature_names_out())
# we get same number of rows but 31260 cols, representing 31260 unique combo of two words
# these are the features we'll be looking at 

### Apply CountVectorizer (w/ N-Grams) to smaller sample

In [ ]:
data_sample = data[0:20]

ngram_vec_sample = CountVectorizer(ngram_range=(2,2))
x_counts_sample = ngram_vec_sample.fit_transform(data_sample['cleaned_text'])

# and just like before:
print(x_counts_sample.shape)
print(ngram_vec_sample.get_feature_names_out())

### Vectorizers output sparse matrices

_**Sparse Matrix**: A matrix in which most entries are 0. In the interest of efficient storage, a sparse matrix will be stored by only storing the locations of the non-zero elements._

In [ ]:
# same as last tiem, we need to convert the data
x_counts_df = pd.DataFrame(x_counts_sample.toarray())
x_counts_df.columns = ngram_vec_sample.get_feature_names_out()
x_counts_df